In [1]:
%pip install -q flwr[simulation] 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda

In [2]:
import os
import IPython.display as display_output
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, Layout
import ipywidgets as widgets
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
from collections import OrderedDict
from typing import List, Tuple
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import flwr as fl
global classes, trainloaders, valloaders, testloader, model, num_clients, num_c

2024-05-07 01:28:38.048645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 01:28:38.048761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 01:28:38.206040: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 01:28:48,302	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
size_label = widgets.Label()

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Required for Kaggle to work with PyTorch
DEVICE = torch.device("cuda")
torch.cuda.is_available()  # Check if GPU is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__}"
)

Training on cuda using PyTorch 2.1.2


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


In [9]:
global num_c, trainloaders, valloaders, testloader, model, classes
num_clients = 15
# Dataset paths
dataset_paths = '/kaggle/input/pneumonia/chest_xray'

datatrain = f'{dataset_paths}/train'
datatset = f'{dataset_paths}/test'
BATCH_SIZE = 32
img_size = (224, 224)

# Download and transform 
transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

trainset = ImageFolder(datatrain, transform=transform)
testset = ImageFolder(datatset, transform=transform)
size_label.value = f'The size of data: {len(trainset) + len(testset)}'
print(size_label)
# Get the classes from the trainset
classes = trainset.classes
num_c = len(classes)

# Split training set into num_clients partitions to simulate the individual dataset
partition_size = len(trainset) // num_clients
remainder = len(trainset) % num_clients
lengths = [partition_size + remainder] + [partition_size] * (num_clients - 1)
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    print("hi")
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x


# Function to train the network
def train(net, trainloader,valloader, epochs: int,patience: int = 5, verbose=True):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    best_val_acc = 0.0
    early_stop_counter = 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        val_loss, val_acc = test(net, valloader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping after {epoch + 1} epochs.")
                break

        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}, val loss {val_loss}, val accuracy {val_acc}")

# Function to test the network
def test(net, testloader,verbose=True):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

# Create the neural network model
model = Net(num_c).to(DEVICE)
print("Number of classes:", num_c)
print('num_clients', num_clients)
print('Waiting for the plot to display...')



# Create a dictionary to store one image from each class
class_images = {}

# Iterate over the trainloaders to get one image from each class
for trainloader in trainloaders:
    for images, labels in trainloader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        for image, label in zip(images, labels):
            class_name = classes[label]
            if class_name not in class_images:
                # Reshape and convert the image to a NumPy array
                image = image.permute(1, 2, 0).cpu().numpy()
                # Denormalize
                image = image / 2 + 0.5
                class_images[class_name] = image
                break

# Create a figure and a grid of subplots
fig, axs = plt.subplots(1, len(class_images), figsize=(5, 3))

# Loop over the classes and plot the corresponding image
for i, (class_name, image) in enumerate(class_images.items()):
    ax = axs[i]
    ax.imshow(image)
    ax.set_title(class_name)
    ax.axis("off")

# Set the title of the plot
fig.suptitle('Sample from selected classes', fontsize=14)

# Show the plot
plt.tight_layout()
plt.rcParams['figure.dpi'] = 100

# Create an output widget
output_widget = widgets.Output()

# Display the plot inside the output widget
with output_widget:
    plt.show()

# Display the output widget
display_output.display(output_widget)

Label(value='The size of data: 5840')
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
Number of classes: 2
num_clients 15
Waiting for the plot to display...


Output()

In [10]:
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__}"
)
print(len(trainloaders))

input_num_epochs = 15
trainloader = trainloaders[0]  # Select the first trainloader for centralized training
valloader = valloaders[0]  # Select the first valloader for centralized training
accuracy_history = []

# Pass valloader as an additional argument to train
train(model.to(DEVICE), trainloader, valloader, input_num_epochs)
loss, accuracy = test(model.to(DEVICE), testloader)
accuracy_history.append(accuracy)
print(loss,accuracy)

message = 'training is over'
display(HTML(message))

Training on cuda using PyTorch 2.1.2
15
Epoch 1: train loss 0.023688791319727898, accuracy 0.718266253869969, val loss 0.02370721697807312, val accuracy 0.8285714285714286
Epoch 2: train loss 0.010603299364447594, accuracy 0.891640866873065, val loss 0.01258035408599036, val accuracy 0.8857142857142857
Epoch 3: train loss 0.009114887565374374, accuracy 0.9164086687306502, val loss 0.005259151543889727, val accuracy 0.9428571428571428
Epoch 4: train loss 0.004208031110465527, accuracy 0.9566563467492261, val loss 0.007859175545828684, val accuracy 0.9428571428571428
Epoch 5: train loss 0.0042364527471363544, accuracy 0.9473684210526315, val loss 0.003172984260267445, val accuracy 0.9714285714285714
Epoch 6: train loss 0.0019871860276907682, accuracy 0.9721362229102167, val loss 0.005182073786688436, val accuracy 0.9428571428571428
Epoch 7: train loss 0.0014374109450727701, accuracy 0.9814241486068112, val loss 0.008680086187086999, val accuracy 0.9142857142857143
Epoch 8: train loss 0.0